In [41]:
#data analysis libraries 
import numpy as np
import pandas as pd

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [42]:
train = pd.read_csv("/home/surya/Downloads/titanic/train.csv")
test = pd.read_csv("/home/surya/Downloads/titanic/test.csv")

In [43]:
Frames = [train,test]
df = pd.concat(Frames)

In [44]:
test.head().T

,0,1,2,3,4
PassengerId,892,893,894,895,896
Pclass,3,3,2,3,3
Name,"Kelly, Mr. James","Wilkes, Mrs. James (Ellen Needs)","Myles, Mr. Thomas Francis","Wirz, Mr. Albert","Hirvonen, Mrs. Alexander (Helga E Lindqvist)"
Sex,male,female,male,male,female
Age,34.5,47.0,62.0,27.0,22.0
SibSp,0,1,0,0,1
Parch,0,0,0,0,1
Ticket,330911,363272,240276,315154,3101298
Fare,7.8292,7.0,9.6875,8.6625,12.2875
Cabin,NaN,NaN,NaN,NaN,NaN


In [45]:
train.describe(include = 'all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [46]:
print(pd.isnull(df).sum())

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64


In [47]:

df = df.drop(['Ticket'], axis = 1)


In [48]:
df['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [49]:
#no of people embarking from Southampton
df[df['Embarked']=='S'].shape[0]

914

In [50]:
#no of people embarking from Cherbourg
df[df['Embarked']=='C'].shape[0]

270

In [51]:
#no of people embarking from Queenstown
df[df['Embarked']=='Q'].shape[0]

123

In [52]:
df = df.fillna({"Embarked": "S"})

In [53]:
combine = [train, test]

#extract a title for each Name in the train and test datasets
for dataset in df:
    df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(df['Title'], df['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,4
Countess,1,0
Don,0,1
Dona,1,0
Dr,1,7
Jonkheer,0,1
Lady,1,0
Major,0,2


In [54]:
for dataset in df:
    df['Title'] = df['Title'].replace(['Lady', 'Capt', 'Col',
    'Don', 'Dr', 'Major', 'Rev', 'Jonkheer', 'Dona'], 'Rare')
    
    df['Title'] = df['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.285714
5,Royal,1.000000


In [55]:
#map each of the title groups to a numerical value
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Royal": 5, "Rare": 6}
for dataset in df:
    df['Title'] = df['Title'].map(title_mapping)
    df['Title'] = df['Title'].fillna(0)


In [56]:
df = df.drop(['Name'], axis = 1)


In [57]:
sex_mapping = {"male": 0, "female": 1}
df['Sex'] = df['Sex'].map(sex_mapping)




In [58]:
embarked_mapping = {"S": 1, "C": 2, "Q": 3}
df['Embarked'] = df['Embarked'].map(embarked_mapping)




In [59]:
df = df.drop(['Cabin'], axis = 1)


In [60]:
result = df.groupby('Pclass')['Age'].agg(lambda x: x.mode().iat[0])

# print the result
print(result)

Pclass
1    36.0
2    24.0
3    22.0
Name: Age, dtype: float64


In [61]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0.0,3,0,22.0,1,0,7.2500,1,0.0
1,2,1.0,1,1,38.0,1,0,71.2833,2,0.0
2,3,1.0,3,1,26.0,0,0,7.9250,1,0.0
3,4,1.0,1,1,35.0,1,0,53.1000,1,0.0
4,5,0.0,3,0,35.0,0,0,8.0500,1,0.0
...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,0,NaN,0,0,8.0500,1,0.0
414,1306,NaN,1,1,39.0,0,0,108.9000,2,0.0
415,1307,NaN,3,0,38.5,0,0,7.2500,1,0.0
416,1308,NaN,3,0,NaN,0,0,8.0500,1,0.0


In [64]:
df = df.reset_index()

In [65]:
def fillna_with_mode(dataf, cols):
    for col in cols:
        dataf[col] = dataf.groupby(['Pclass'])[col].apply(lambda x: x.fillna(x.mode()[0]))
     
    
    return dataf


In [66]:
df = fillna_with_mode(df, ['Age'])


In [67]:
df['Survived'] = df['Survived'].fillna(1)

In [73]:
from sklearn.model_selection import train_test_split

predictors = df.drop(['Survived', 'PassengerId','Fare','Title'], axis=1)
target = df["Survived"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 0)

In [74]:
print(pd.isnull(predictors).sum())

index       0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64


In [27]:
predictors

,index,Pclass,Sex,Age,SibSp,Parch,Embarked,Title
0,0,3,0,22.0,1,0,1,0.0
1,1,1,1,38.0,1,0,2,0.0
2,2,3,1,26.0,0,0,1,0.0
3,3,1,1,35.0,1,0,1,0.0
4,4,3,0,35.0,0,0,1,0.0
...,...,...,...,...,...,...,...,...
1304,413,3,0,22.0,0,0,1,0.0
1305,414,1,1,39.0,0,0,2,0.0
1306,415,3,0,38.5,0,0,1,0.0
1307,416,3,0,22.0,0,0,1,0.0


In [28]:
#Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

gbk = GradientBoostingClassifier()
gbk.fit(predictors,target)



GradientBoostingClassifier()

In [29]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(gbk, f)

In [75]:
predictors.head()

,index,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,0,22.0,1,0,1
1,1,1,1,38.0,1,0,2
2,2,3,1,26.0,0,0,1
3,3,1,1,35.0,1,0,1
4,4,3,0,35.0,0,0,1
